In [52]:
import numpy as np

# 2. Nussinov

In [159]:
RNA = 'GGGGCUAUAGCUCAGCUGGGAGAGCGCCUGCUUUGCACGCAGGAGGUCUGCGGUUCGAUCCCGCAUAGCUCCACCA' # Escherichia coli tRNA

In [160]:
ind = {'A': 0, 'U': 1, 'G': 2, 'C': 3}
w = [[0,1,0,0], [1,0,1,0], [0,1,0,1], [0,0,1,0]] # матрица комплиментарности пар нуклеотидов

In [161]:
def Nuss(s, w, min_hp=3):     # min_hp - минимальный размер шпильки
    n = len(s)
    M = np.zeros((n,n), dtype=np.int)
    
    for d in range(n-1):     # динамическое заполнение матрицы наилучших вариантов укладки
        i, j = 0, min_hp + d
        while j < n:
            K = np.empty(j-i, dtype=np.int)
            for k in range(i,j):
                K[i-k] = M[i,k] + M[k+1,j]
            M[i,j] = max(M[i+1,j-1] + w[ind[s[i]]][ind[s[j]]] ,np.max(K))
            i += 1
            j += 1
            
    stack = [(0,n-1)]
    pairs = []
    while len(stack):     # восстановление решения
        i, j = stack.pop()
        if j-i <= min_hp:
            continue
        elif M[i,j] == M[i+1, j-1] + w[ind[s[i]]][ind[s[j]]]:
            pairs.append((i,j))
            stack.append((i+1, j-1))
        else:
            for k in range(i,j):
                if M[i,j] == M[i,k] + M[k+1,j]:
                    stack.append((i, k))
                    stack.append((k+1,j))
                    break
    pairs.sort()
    struct = ['-']*n     # представление решения
    for i, j in pairs: 
        struct[i] = '('
        struct[j] = ')'
        
    print(s)
    print(*struct, sep='')
    
    return M[0,-1]

In [162]:
Nuss(RNA, w)

GGGGCUAUAGCUCAGCUGGGAGAGCGCCUGCUUUGCACGCAGGAGGUCUGCGGUUCGAUCCCGCAUAGCUCCACCA
(((((----))))--)((((-(((((((-((((((----))))(----))))))(-(-(-----))))))))-)))


30

Попробуем использовать разные веса для разных пар нуклеотидов

In [169]:
w1 = [[0,2,0,0], [2,0,1,0], [0,1,0,3], [0,0,3,0]] # разные веса для разных пар нуклеотидов

Nuss(RNA, w1)
print()

GGGGCUAUAGCUCAGCUGGGAGAGCGCCUGCUUUGCACGCAGGAGGUCUGCGGUUCGAUCCCGCAUAGCUCCACCA
(((((----))))--)((((-((((((((-(((-(----)))))))-)(((((-------))))--))))))-)))



### Test 1

In [166]:
Nuss('GGACC', w)

GGACC
(---)


1

### Test 2

In [167]:
Nuss('AAACAUGAGGAUUACCCAUGU', w)

AAACAUGAGGAUUACCCAUGU
(--((((((-----)))))))


7